# Vector Search Interaction with Patient Files

Patient records can be tricky, and navigating them can be tricky in healthcare. Nevertheless, it is crucial to be able to do so.

In theory, this is done with meticulous standards and making sure there is very clear terminology in symptoms and observations. In practice, this is not always the case. For doctors seeing a patient that has a lengthy record for the first time it can be hard to decipher the relevant parts of the file. Add inconsistent word use and suddently standard search functions become very difficult to use.

Using Vector Search and Embeddings, we can hope to fix this issue. By embedding the semantic meaning of a word, we no longer need to rely on exact word use, and other relevant reports can come through the search to enrich the search result.

To showcase this use case, I have made an AI agent that lets you filter for relevant parts of patient files from a public dataset. We will ask it things like "Has this patient shown signs of dementia before?" and be able to get results back of reports that might be relevant. Should, for example, there be an increase of dementia-related reports over the past 6 months or so, a medical expert might want to act on that.

As a technical challenge, I wanted to do this entire process with the BigQuery Python library.

# Data setup
Using a public dataset of fake patient data, we can extract patient reports from 20 ficticious patients.
The setup of the dataset is a bit limited, but we will further enrich that later. 

People to process is set to 10 by default, but could in theory be bumped up if you're prepared to foot the bill.
Similarly the reports per patient is limited; there are thousands of patients in this dataset each with hundreds of reports!

In [1]:
!pip install --quiet google-cloud-bigquery-storage #Reduces future errors and increases readability

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [2]:

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
PROJECT_ID = user_secrets.get_secret("GCP_PROJECT_ID")
DATASET_ID = 'concept_data'
DATASET_TABLE = 'concept_content'
DATASET_LOCATION = 'US' 

PATIENTS_TO_PROCESS = 10 #How many patients to grab from the public dataset
REPORTS_PER_PATIENT = 50 #How many reports do you want to generate and embed per person?

print('Project ID and Database settings have been succesfully set.')

Project ID and Database settings have been succesfully set.


First, we generate an empty BigQuery Dataset to later create our tables and models into.
I default to setting the location to US, as newer models tend to be available there slightly earlier than in my area (Europe).

In [3]:
from google.cloud import bigquery
from google.api_core import exceptions as api_exceptions

#BigQuery Client
client = bigquery.Client(project=PROJECT_ID)

try:
    #Basic Dataset params
    dataset_ref = client.dataset(DATASET_ID)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = DATASET_LOCATION
    
    #Send the dataset to the API for creation.
    dataset = client.create_dataset(dataset)
    print(f"Dataset '{DATASET_ID}' created successfully in {DATASET_LOCATION}")
except api_exceptions.Conflict:
    #Handle the case where the dataset already exists.
    print(f"Dataset '{DATASET_ID}' already exists. Skipping creation.")
except Exception as e:
    #Handle other potential API errors.
    print(f"An error occurred: {e}")

Dataset 'concept_data' created successfully in US


Now that the empty dataset has been generated, we can put our own data into it.

This query takes the public dataset with patient reports, and extracts people and their associated reports.
The dataset has a lot of tables, most of which frankly lacked the context for me to understand, but I grabbed the "concepts" as well as their start and end-date.
This gives us the following setup;
- The patient (person)'s id
- The ID of the condition, and its ancestor (type of condition)
- The start and end date of the condition
- The content of the condition and its ancestor, combined into 1 string
- 2 empty columns, to be used later
- A UUID using BigQuery's FARM_FINGERPRINT function

In [4]:
#Import necessary libraries for BigQuery and data handling
from google.cloud import bigquery
import pandas as pd

#BigQuery Client
client = bigquery.Client(project=PROJECT_ID)

#Query setup, joining tables together from the public dataset to create our own. Lacks a unique identifier, so we use FARM_FINGERPRINT to make one.
# This query will create or replace the 'DATASET_TABLE' table in your dataset.
query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}` AS
WITH RankedConditions AS (
    SELECT
        t1.person_id,
        t1.condition_concept_id,
        t3.ancestor_concept_id,
        ANY_VALUE(t1.condition_start_date) AS condition_start_date,
        ANY_VALUE(t1.condition_end_date) AS condition_end_date,
        FORMAT('Condition Name: %s\\nAncestor Name: %s',
               ANY_VALUE(t2.concept_name),
               ANY_VALUE(t4.concept_name)) AS concept_details,
        CAST(NULL AS STRING) AS generated_report,
        CAST(NULL AS ARRAY<FLOAT64>) AS report_embeddings,
        ROW_NUMBER() OVER(PARTITION BY t1.person_id ORDER BY ANY_VALUE(t1.condition_start_date)) AS row_num
    FROM
        `bigquery-public-data.cms_synthetic_patient_data_omop.condition_occurrence` AS t1
    JOIN
        (SELECT DISTINCT person_id FROM `bigquery-public-data.cms_synthetic_patient_data_omop.person` ORDER BY person_id LIMIT {PATIENTS_TO_PROCESS}) AS limited_persons
        ON t1.person_id = limited_persons.person_id
    JOIN
        `bigquery-public-data.cms_synthetic_patient_data_omop.concept` AS t2
        ON t1.condition_concept_id = t2.concept_id
    JOIN
        `bigquery-public-data.cms_synthetic_patient_data_omop.concept_ancestor` AS t3
        ON t1.condition_concept_id = t3.descendant_concept_id
    JOIN
        `bigquery-public-data.cms_synthetic_patient_data_omop.concept` AS t4
        ON t3.ancestor_concept_id = t4.concept_id
    GROUP BY
        t1.person_id,
        t1.condition_concept_id,
        t3.ancestor_concept_id
)
SELECT
    FARM_FINGERPRINT(TO_JSON_STRING(STRUCT(person_id, condition_concept_id, ancestor_concept_id))) AS unique_id,
    person_id,
    condition_concept_id,
    ancestor_concept_id,
    condition_start_date,
    condition_end_date,
    concept_details,
    generated_report,
    report_embeddings
FROM
    RankedConditions
WHERE
    row_num <= {REPORTS_PER_PATIENT};
"""

print(f"Executing BigQuery query to create/replace table '{DATASET_TABLE}'...")
# Run the query
query_job = client.query(query)
# Wait for the query to complete
query_job.result()
print(f"Table '{DATASET_TABLE}' created/replaced successfully.")

#Querying the resulting table to show the data we are working with.
print("\nFetching first 5 rows from the new table for verification...")
query_results_sql = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}`
LIMIT 5;
"""
df_results = client.query(query_results_sql).to_dataframe()

print("First 5 rows of `{DATASET_TABLE}`:")
print(df_results)

Executing BigQuery query to create/replace table 'concept_content'...
Table 'concept_content' created/replaced successfully.

Fetching first 5 rows from the new table for verification...
First 5 rows of `{DATASET_TABLE}`:
              unique_id  person_id  condition_concept_id  ancestor_concept_id  \
0  -4538431163366809327          8                437851              4081007   
1  -1279648889780597621          8                437851              4303380   
2   6505077824634278573          8                437851              4102183   
3   6329260021517612951          8                437851               441840   
4   8015796092246835506          8                437851              4274025   

  condition_start_date condition_end_date  \
0           2008-02-16         2008-02-16   
1           2008-02-16         2008-02-16   
2           2008-02-16         2008-02-16   
3           2008-02-16         2008-02-16   
4           2008-02-16         2008-02-16   

                    

#### Generating fully fletched SOAP reports
Initially I worked with this data, but my results were frankly quite middling.
This is because, as you can see in these first 5 rows; the Condition name and Ancestor Name combine to about... 4 words?
It's hard to embed something meaningful out of this, when it is so tiny and the terms used are quite generic.

To fix this, we will use these keywords to generate our own ficticious patient reports.
For this, I picked SOAP standard reporting - A standard medical protocol that gives incredibly consistent output.
The generated SOAP reports are quite dense and contain a high standard of information, easily beating out the condition/ancestor combination I used earlier.
Later, we will use these SOAP reports to create a proper vector search embedding.

To do this, we will need to to create the model that will generate these reports for us.
Luckily, you can do that with a BigQuery query!

In [5]:
GENERATION_MODEL_NAME = "report_generation" #Set this to what you want the ML model to be named.
GENERATION_MODEL_TYPE = "gemini-2.5-flash-lite" #What model do you want to use.

In [6]:
full_model_name = f"`{PROJECT_ID}.{DATASET_ID}.{GENERATION_MODEL_NAME}`"

model_generation_query = f"""
CREATE OR REPLACE MODEL {full_model_name}
REMOTE WITH CONNECTION DEFAULT
OPTIONS(ENDPOINT = '{GENERATION_MODEL_TYPE}')
"""

try:
    # Execute the query
    job = client.query(model_generation_query)
    job.result()  # Wait for the job to complete
    print("Model created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Model created successfully.


Now that we have a model, we can use it to generate text with ML.GENERATE_TEXT.
Using BigQueryML, we take the condition and ancestor from the data and add it to the CONCAT.

The prompt is inside the CONCAT statement as you can see; it explains the setup of a SOAP report, gives the concept_details, and then Gemini generates them.
We then update our existing database's generated_report column with the output.

With the default settings of 10 people and 50 reports per, there should be 500 reports to generate split into batches of 100.

In [7]:
from google.cloud import bigquery
import time

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

#The update query to call the Gemini model for each row.
#We'll use a `WHERE` clause to process only rows where the report is still NULL.
#This prevents reprocessing the same data if the script is run again.
update_query = f"""
UPDATE `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}` AS t
SET
    t.generated_report = model_output.ml_generate_text_llm_result
FROM
    ML.GENERATE_TEXT(
        MODEL `{PROJECT_ID}.{DATASET_ID}.{GENERATION_MODEL_NAME }`,
        (
            SELECT
                
                t.unique_id AS unique_row_id,
                CONCAT(
                    "Generate a detailed and fictional patient progress note using the SOAP (Subjective, Objective, Assessment, Plan) format for the following patient data. ",
                    "Do not include any personal identifying information. Keep the tone informative and formal. Do not include date or time, they are noted elsewhere - only report the details of SOAP, and nothing else. ",
                    "Patient Data: ", t.concept_details,
                    "\\nSubjective Based on the patient's condition, describe the patient's chief complaint or subjective narrative of their symptoms.",
                    "\\nObjective Based on the patient's condition, provide an objective description of the findings, such as physical examination results or lab work (create fictional but plausible details).",
                    "\\nAssessment Formulate a differential diagnosis or an assessment of the patient's condition based on the subjective and objective information.",
                    "\\nPlan Outline a plan for the patient's treatment, follow-up, and further testing."
                ) AS prompt
            FROM
                `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}` AS t
            WHERE
                t.generated_report IS NULL
            LIMIT 100
        ),
        STRUCT(
            1024 AS max_output_tokens,
            0.5 AS temperature,
            TRUE AS flatten_json_output
        )
    ) AS model_output
WHERE
    t.unique_id = model_output.unique_row_id;
"""

print(f"Executing update query using BigQuery ML model '{GENERATION_MODEL_NAME }'...")

#Loop until all rows have been updated
while True:
    #Run the update query
    query_job = client.query(update_query)

    #Wait for the query to complete
    query_job.result()

    #Get the number of rows affected by the UPDATE statement
    rows_affected = query_job.num_dml_affected_rows

    print(f"\nUpdate completed. {rows_affected} reports generated and saved.")

    #If no rows were affected, we have processed all of them.
    if rows_affected == 0:
        print("All records have been updated. The process is complete.")
        break

    #Pause briefly between loops to avoid rate-limiting issues.
    time.sleep(1)

print("\nFetching first 5 rows with the new generated reports for verification...")

verification_query = f"""
SELECT
    person_id,
    concept_details,
    generated_report
FROM `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}`
WHERE generated_report IS NOT NULL
LIMIT 5;
"""

df_results = client.query(verification_query).to_dataframe()

print("First 5 rows of `concept_content` with generated reports:")
print(df_results)

Executing update query using BigQuery ML model 'report_generation'...

Update completed. 100 reports generated and saved.

Update completed. 100 reports generated and saved.

Update completed. 100 reports generated and saved.

Update completed. 100 reports generated and saved.

Update completed. 100 reports generated and saved.

Update completed. 0 reports generated and saved.
All records have been updated. The process is complete.

Fetching first 5 rows with the new generated reports for verification...
First 5 rows of `concept_content` with generated reports:
   person_id                                    concept_details  \
0          8  Condition Name: Open angle with borderline int...   
1          8  Condition Name: Open angle with borderline int...   
2          8  Condition Name: Open angle with borderline int...   
3          8  Condition Name: Open angle with borderline int...   
4          8  Condition Name: Open angle with borderline int...   

                             

We now have all our data! And its quite bolsterous too!

# Embedding our reports

For those unfamiliar, with Embeddings we 'vectorize' the words; this lets us store the meaning of a word rather than just the letters itself, which in turn lets you query for them later; vectorizing the word 'phone' will store it alongside similar words like 'telephone', 'mobile phone', or maybe even some phone brands. In a medical context, we're basically grouping similar reports together. 
When then asking a question, we will scoop up reports that are very close to the embedded version of what we are asking, thus getting only relevant reports back.

Like before, we need to generate an ML model for this.
I'll be using Google's own text-embedding model.

In [8]:
EMBEDDING_MODEL_NAME = "embedding_generation" #Set this to what you want the ML model to be named.
EMBEDDING_MODEL_TYPE = "text-embedding-005" #What model do you want to use.

In [9]:
full_model_name = f"`{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_NAME}`"

model_generation_query = f"""
CREATE OR REPLACE MODEL {full_model_name}
REMOTE WITH CONNECTION DEFAULT
OPTIONS(ENDPOINT = '{EMBEDDING_MODEL_TYPE}')
"""

try:
    # Execute the query
    job = client.query(model_generation_query)
    job.result()  # Wait for the job to complete
    
    print("Model created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Model created successfully.


Now that we have our model, we can create embeddings.
For each row of our dataset, the content gets turned into embeddings and written back to our table. We can use the dedicated ML.GENERATE_EMBEDDING for this



In [10]:
from google.cloud import bigquery
import time

# Initialize the BigQuery client
client = bigquery.Client(project=PROJECT_ID)

merge_query = f"""
MERGE `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}` AS T
USING (
  WITH ReportsToEmbed AS (
    SELECT
      t.unique_id,
      t.generated_report AS content,
      ROW_NUMBER() OVER() AS row_num
    FROM
      `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}` AS t
    WHERE
      t.generated_report IS NOT NULL AND (t.report_embeddings IS NULL OR ARRAY_LENGTH(t.report_embeddings) = 0)
    LIMIT 100
  ),
  EmbeddingsWithId AS (
    SELECT
      ml_generate_embedding_result,
      ROW_NUMBER() OVER() AS row_num
    FROM
      ML.GENERATE_EMBEDDING(
        MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_NAME}`,
        (SELECT content FROM ReportsToEmbed ORDER BY row_num)
      )
  )
  SELECT
    r.unique_id,
    e.ml_generate_embedding_result
  FROM
    ReportsToEmbed r
  JOIN
    EmbeddingsWithId e
  ON
    r.row_num = e.row_num
) AS S
ON T.unique_id = S.unique_id
WHEN MATCHED THEN
  UPDATE SET T.report_embeddings = S.ml_generate_embedding_result;
"""

print(f"Embedding reports using the BigQuery ML model  and updating the '{DATASET_TABLE}' table...")

# Loop until all reports have been embedded
while True:
    #Run the MERGE query
    query_job = client.query(merge_query)

    #Wait for the query to complete
    query_job.result()

    #Get the number of rows affected by the MERGE statement
    rows_affected = query_job.num_dml_affected_rows

    print(f"\nMerge completed. {rows_affected} reports embedded and saved.")

    # If no rows were affected, all reports have been processed.
    if rows_affected == 0:
        print("All records have been embedded. The process is complete.")
        break

    # Optional: Pause briefly between loops to avoid rate-limiting issues.
    time.sleep(1)

print("\nFetching first 5 rows with the new embeddings for verification...")

verification_query = f"""
SELECT
    person_id,
    generated_report,
    report_embeddings
FROM `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}`
WHERE report_embeddings IS NOT NULL
LIMIT 5;
"""

df_results = client.query(verification_query).to_dataframe()

print("First 5 rows of `concept_content` with generated embeddings:")
print(df_results)

Embedding reports using the BigQuery ML model  and updating the 'concept_content' table...

Merge completed. 100 reports embedded and saved.

Merge completed. 100 reports embedded and saved.

Merge completed. 100 reports embedded and saved.

Merge completed. 100 reports embedded and saved.

Merge completed. 100 reports embedded and saved.

Merge completed. 0 reports embedded and saved.
All records have been embedded. The process is complete.

Fetching first 5 rows with the new embeddings for verification...
First 5 rows of `concept_content` with generated embeddings:
   person_id                                   generated_report  \
0          5  **Subjective:**\nPatient reports persistent, d...   
1          5  Subjective:\nPatient reports a gradual onset o...   
2          5  **Subjective:**\nPatient reports a progressive...   
3          5  **Subjective:**\nPatient reports a persistent,...   
4          5  **Subjective:**\nPatient reports persistent, d...   

                       

We've not successfully embedded the content!
As you can see by the output, the report_embeddings are now entirely unreadable for humans but it is key to perform a similarity search.

# Querying our Embeddings.


As a proof of concept, we will now query a symptom against our data. Because our reports are done per person, we can query it for only 1 patient and see what reports are similar to what we asked.
There are multiple ways to do this, but to stay on theme here I want to keep using BigQuery for it and use VECTOR_SEARCH function (I did initially also use ML.DISTANCE)

For a more Google-minded approach, look into Vector Search: https://cloud.google.com/vertex-ai/docs/vector-search/overview - This, in my opnion, only gets interesting with huge datasets.
My preferred method is using the sklearn python library, however I have not done that at scale.

On top of the similarity search, I added a minimal_similarity treshold - this is to not waste the caretakers time by giving irrelevant results back. This is important in any and all Vector Searches, as the code as is will always try to return the highest ranking results even if they barely match at all.


In [11]:
query = "Lower back problems"
patient_id = 1
start_date = "2009-01-01"
end_date = "2010-01-01"

In [12]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

TOP_K_RESULTS = 3 #How many results we want returned
MINIMAL_SIMILARITY = 0.6 #How good results have to be in order to be returned. 

def perform_similarity_search_bigquery_ml(query, patient_id, start_date, end_date):
    client = bigquery.Client(project=PROJECT_ID)

    try:
        print(f"Performing similarity search using BigQuery ML for query: '{query}'")

        # The SQL query to generate the query embedding and perform the search
        sql_query = f"""
        SELECT
          base.concept_details,
          base.generated_report,
          1 - distance AS cosine_similarity -- Convert cosine distance to similarity
        FROM
          VECTOR_SEARCH(
            (
              SELECT
                *
              FROM
                `{PROJECT_ID}.{DATASET_ID}.{DATASET_TABLE}`
              WHERE
                person_id = @patient_id
                AND condition_start_date >= @start_date
                AND condition_end_date < @end_date
            ),
            'report_embeddings', -- The column to search, as a string literal
            (
              SELECT
                ml_generate_embedding_result AS embedding
              FROM
                ML.GENERATE_EMBEDDING(
                  MODEL `{PROJECT_ID}.{DATASET_ID}.{EMBEDDING_MODEL_NAME}`,
                  (SELECT @query AS content)
                )
            ),
            query_column_to_search => 'embedding', -- Named argument for the query column
            top_k => @top_k_results, -- Named argument for top_k
            distance_type => 'COSINE' -- Named argument for distance_type
          )
        WHERE
          1 - distance >= @minimal_similarity 
        ORDER BY
          cosine_similarity DESC;
        """
        
        #Configure the query with parameters to prevent SQL injection and ensure type safety
        job_config = bigquery.QueryJobConfig(
            query_parameters=[
                bigquery.ScalarQueryParameter("query", "STRING", query),
                bigquery.ScalarQueryParameter("patient_id", "INT64", patient_id),
                bigquery.ScalarQueryParameter("start_date", "STRING", start_date),
                bigquery.ScalarQueryParameter("end_date", "STRING", end_date),
                bigquery.ScalarQueryParameter("top_k_results", "INT64", TOP_K_RESULTS),
                bigquery.ScalarQueryParameter("minimal_similarity", "FLOAT64", MINIMAL_SIMILARITY),

            ]
        )

        #Run the query
        query_job = client.query(sql_query, job_config=job_config)

        #Load the results directly into a DataFrame
        results_df = query_job.to_dataframe()

        if results_df.empty:
            print("No matching concepts found for the given criteria.")
            return

        # Display the results
        print(f"\n--- Top Matching Concepts for query: '{query}' ---")
        for rank, row in results_df.iterrows():
            print(f"\nRank {rank + 1}: Score = {row['cosine_similarity']:.4f}")
            print(f"  Concept: {row['concept_details']}")
            print(f"  Details: {row['generated_report']}")
            print("------------")

    except Exception as e:
        print(f"An error occurred: {e}")

perform_similarity_search_bigquery_ml(query=query, patient_id=patient_id, start_date=start_date, end_date=end_date)

Performing similarity search using BigQuery ML for query: 'Lower back problems'

--- Top Matching Concepts for query: 'Lower back problems' ---

Rank 1: Score = 0.6961
  Concept: Condition Name: Low back pain
Ancestor Name: Backache
  Details: **Subjective:**
Patient reports a chief complaint of persistent low back pain, described as a dull, aching sensation that has been present for approximately three weeks. The pain is localized to the lumbar region and occasionally radiates into the posterior thighs bilaterally, but does not extend below the knees. The patient reports the pain is exacerbated by prolonged sitting, standing, and bending forward. They deny any recent trauma or injury. Pain intensity is rated as a 6/10 at its worst and 2/10 at its best. The patient reports mild stiffness in the morning that improves with movement. They deny any bowel or bladder incontinence, numbness, or tingling in the extremities, or fever.

**Objective:**
Physical examination reveals a well-appearin

Like said before, we are keeping with the theme and using BigQueryML.

A bit hard to describe as it's 1 big SQL statement, but we are doing the following:
- Generating Embeddings for our Query, the same way we have done for our generated reports before. This makes them comparable to each other as they use the same embeddings/language.
- Using VECTOR_SEARCH to compare this embedding to the embeddings from our table.
- This performs a cosine similarity search to see how "similar" they are.
- The top 3 reports then get returned to us.

With the default values, I know patient 1 suffers from lower back problems by looking through the data prior.
Indeed, the highest returning scores from our vector search for our question relates to their back issues. 

# Creating an Agent to call our function
This is all well and good, but this isn't usable for a medical professional. At least, the ones I've met aren't proficient in Python and BigQuery.

Using Gemini, we can interact with it and have it call our python functions when it deems appropiate.
This is done using a concept called function calling - https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling

In the following code we specify a prompt and a patient to ask it about.
We then create a Tool; this is the format that Gemini uses to do Function Calling.
Assuming we ask a good enough question, Gemini will then recognize it and fill in the function arguments for us.

The code then executes the function with the params given by Gemini.

In [13]:
MODEL_NAME = "gemini-2.5-flash-lite"
PROMPT = "Does this patient have a history with lower back problems?"
PATIENT_ID = 1

In [14]:
import time
import json
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    Tool,
)

from google.cloud import bigquery
import time

from kaggle_gcp import KaggleKernelCredentials

try:
    # Get the credentials object from the Kaggle environment
    credentials = KaggleKernelCredentials()

    # Pass the credentials object directly to the genai.Client
    client = genai.Client(
        vertexai=True,
        project=PROJECT_ID,
        location="us-east1",
        credentials=credentials 
    )
    print("GenAI client initialized successfully with Kaggle credentials.")
except Exception as e:
    print(f"Failed to initialize client with credentials: {e}")
    raise SystemExit("Exiting due to authentication failure.")

def function_calling_agent():
    #Define the tool for the model using the FunctionDeclaration and Tool classes.
    get_similarity_search = FunctionDeclaration(
        name='perform_similarity_search_bigquery_ml',
        description='Performs a semantic similarity search on the medical report data to find patient reports that are similar to the question of the medical professional interacting with you.',
        
        #Function parameters are specified in JSON schema format
        parameters={
            "type": "OBJECT",
            "properties": {
                'query': {"type": "STRING", "description": 'The user\'s search query.'},
                'patient_id': {"type": "STRING", "description": 'The ID of the patient.'},
                'start_date': {"type": "STRING", "description": 'The start date of the search range in YYYY-MM-DD format. If not given, assume 1970-01-01'},
                'end_date': {"type": "STRING", "description": 'The end date of the search range in YYYY-MM-DD format. If not given, assume today'},
            },
            "required": ['query', 'patient_id', 'start_date', 'end_date'],
        },
    )

    #Defining the tool itself to use the function
    search_tool = Tool(function_declarations=[get_similarity_search])

    #Create a user prompt that would naturally trigger the function call.
    final_prompt = PROMPT + f"\nPatient ID: {PATIENT_ID}, current date: {time.strftime('%Y-%m-%d')}" #Adding our other variables to the prompt.
    print("Sending prompt to the model...")

    #Generating the content now with all of our input. Temperature lets Gemini be a bit 'creative' with its output.
    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=final_prompt,
        config=GenerateContentConfig(
            tools=[search_tool],
            temperature=0.3,
        ),
    )

    #Check the response for a function call and execute it.
    if response.function_calls:
        print("\nModel response contains a function call:")
        call = response.function_calls[0]
        print(call)

        #Get the Python functio to call
        function_to_call = globals()[call.name]

        #Execute the function with the arguments provided by the model.
        function_result = function_to_call(**call.args)
    else:
        print("\nModel did not return a function call.")
        print(f"Model response: {response.text}")


function_calling_agent()

GenAI client initialized successfully with Kaggle credentials.
Sending prompt to the model...

Model response contains a function call:
id=None args={'start_date': '1970-01-01', 'end_date': '2025-09-18', 'query': 'lower back problems', 'patient_id': '1'} name='perform_similarity_search_bigquery_ml'
Performing similarity search using BigQuery ML for query: 'lower back problems'

--- Top Matching Concepts for query: 'lower back problems' ---

Rank 1: Score = 0.6915
  Concept: Condition Name: Low back pain
Ancestor Name: Backache
  Details: **Subjective:**
Patient reports a chief complaint of persistent low back pain, described as a dull, aching sensation that has been present for approximately three weeks. The pain is localized to the lumbar region and occasionally radiates into the posterior thighs bilaterally, but does not extend below the knees. The patient reports the pain is exacerbated by prolonged sitting, standing, and bending forward. They deny any recent trauma or injury. Pain 

As you can (hopefully) see, patient 1 does indeed have a history with back problems like we've seen earlier in our BigQueryML function.
Let's try 1 more.

In [15]:
PROMPT = "Did this patient have issues with diabetes through 2009?"
PATIENT_ID = 3

function_calling_agent()

Sending prompt to the model...

Model response contains a function call:
id=None args={'query': 'diabetes', 'end_date': '2009-12-31', 'patient_id': '3', 'start_date': '2009-01-01'} name='perform_similarity_search_bigquery_ml'
Performing similarity search using BigQuery ML for query: 'diabetes'

--- Top Matching Concepts for query: 'diabetes' ---

Rank 1: Score = 0.6292
  Concept: Condition Name: Type 2 diabetes mellitus
Ancestor Name: Type 2 diabetes mellitus
  Details: **Subjective:**
Patient reports persistent polydipsia and polyuria over the past several weeks. They also endorse increased fatigue and occasional blurred vision. Patient denies any recent changes in diet or exercise habits. No reported history of diabetic ketoacidosis or hyperosmolar hyperglycemic state. Patient expresses concern about their energy levels and potential long-term complications.

**Objective:**
Vital Signs: Blood pressure 138/88 mmHg, heart rate 78 bpm, respiratory rate 16 bpm, temperature 98.6°F.
Physic

And 1 final application;

In [16]:
PROMPT = "Have there been dementia related symptoms since september 2009?"
PATIENT_ID = 4

function_calling_agent()

Sending prompt to the model...

Model response contains a function call:
id=None args={'patient_id': '4', 'query': 'dementia related symptoms', 'start_date': '2009-09-01', 'end_date': '2025-09-18'} name='perform_similarity_search_bigquery_ml'
Performing similarity search using BigQuery ML for query: 'dementia related symptoms'
No matching concepts found for the given criteria.


As you can see, no results. The similarity search tresholds filters them - in reality, this patient has only suffered from spinal issues and diabetes in this window.

If we were to say, deploy this to a Cloud Run (or Google's Agent Engine), we would now have a REST API Endpoint that can be called by any application to actually Vector Search through patient data!

# Conclusions

Hopefully I've been able to showcase the power of BigQueryML here. I was impressed that all of this functionality works within SQL; I was worried that this would have to be a video showcase of me going through bigquery and setting up all these things by hand (like for example, creating the Generation and Embedding Models), but luckily this was all possible with just BigQuery calls inside Kaggle. The speed of the BigQueryML calls also surprised me, especially the similarity search.

I'm a big fan of embeddings in the medical sphere. I've worked in healthcare-related IT for a few years and the amount of administration and bulk that these nurses and other professionals need to go through is absurd. Hopefully we can, in the near future, assist them in this proces. The quality of our models will keep improving, and I think this can be a reality soon.

The lack of consistant language use inbetween organisations and programs can cause extra bloat in this administrative process. Embeddings and Classifications within GCP can realy alleviate these issues.

If I were to do this again, I would set up something more robust for the agent using Google's Agent Development Kit (ADK). It's elegant use of sub-agents would let you run a host of queries all within the same project while still having a clean project setup. That being said, I'm not sure how it would interact with a Kaggle Notebook and I would find it unrelated to the BigQuery usecase. Should I bring this to production, I will definitely read into it more.

Outside of that, I would write something to reduce the verboseness of the SOAP reports - they're quite wordy, and it troubles the embeddings somewhat. 